In [1]:
import requests
import pandas as pd 
import json
import plotly.graph_objects as go
import numpy as np


### Relative Squawks vs. Relative Diversions

In [2]:
df_squawk = pd.read_csv('../data/processed/squawk7700_processed_final_v2.csv')

In [3]:
# Data prep frequency of IFE
df_ife_diversion = df_squawk.groupby('productionLine').flight_id.count().sort_values(ascending=False).reset_index()
df_ife_diversion.columns = ['productionLine','ife_count']
df_ife_diversion.head()

# Data prep frequency of diversions
df_squawk['diverted'] = (~df_squawk['landing'].isna()) & (df_squawk['landing'] != df_squawk['destination']) & (df_squawk['origin'] != df_squawk['destination'])

diverted_type_df = df_squawk[df_squawk['diverted']==True].groupby('productionLine').flight_id.count().reset_index()
diverted_type_df.rename(columns={'flight_id':'diversion_count'}, inplace=True)
diverted_type_df.sort_values(by='diversion_count',ascending=False, inplace=True)

# Merge both data preps
df_ife_diversion = pd.merge(df_ife_diversion, 
                            diverted_type_df, 
                            on='productionLine', 
                            how='left')

# Plot
fig = go.Figure(data=[
    go.Bar(x=df_ife_diversion['productionLine'], y=df_ife_diversion['ife_count']/df_ife_diversion['ife_count'].sum(), name="In Flight Emergency", marker_color="green"),
    go.Bar(x=df_ife_diversion['productionLine'], y=df_ife_diversion['diversion_count']/df_ife_diversion['diversion_count'].sum(), name="Diversion", marker_color="red")
])

fig.update_layout(
    title={'text': "Normalized Frequencies of IFEs and Diversions", 'font': {"size": 24}},
    xaxis_title="Aircraft Model Family",
    yaxis_title="Normalized Frequencies",
)